# Selecting the examples for analisys
Long, medium and short text entries


In [40]:
import numpy as np
import pandas as pd
import random
from gensim.utils import simple_preprocess

In [41]:
df = pd.read_csv("../../data/ad_hominem/ad_hominems_cleaned_Murilo.csv", sep=",", index_col=0, header=0, names=["body", "isAdHominem"])
df = df[~df.isin([np.nan, np.inf, -np.inf, 'nan']).any(1)] ## Remove rows with NaN values
print(df.shape)

(29218, 2)


In [42]:
df["length"] = df["body"].apply(lambda x: len(simple_preprocess(x, deacc=True)))

In [43]:
pd.set_option('display.max_colwidth', 0)
df.head(n=5)

,body,isAdHominem,length
0,What makes corporations different in this case? They have interests too.,0,11
1,"I'm sorry if your smugness gets in the way. Like I said elsewhere in this thread. Somolia is not close to anything I advocate for so why on earth would I move there? Any time the Somolia ""argument"" is brought up, I instantly know I'm dealing with someone who refuses to learn the difference between a voluntary society and a third world country ravaged by warlords and foreign policies of other countries. If you want a thoughtful response to an argument, make sure you're not comparing Antarctica to the Bahamas. Otherwise, take your circlejerk, ""arguments"" elsewhere. You have contributed absolutely nothing to this thread but ad hominem Attacks and the typical liberal/conservative talking points and almost everyone in here knows it.",1,114
2,"Basically to believe a patriarchy exists, you must believe that men are maintaining a system of oppression against women, despite knowing the harm it does to both women and men.EG - Wanting to maintain a system that, among other things, condones severe anti-male bias in all facets of the legal system, simply isn't rational. Thus men, being the ones in power, want to oppress women so much they are willing to harm themselves to do it. It'd be like cutting off your own arm so you had something to club someone with.A long time ago one could say it was ignorance, but with how mainstream feminism thoughts are today this can no longer be true. So the actions of men to maintain the patriarchy must also be willful.How can a person believe this, and not hate men?",0,135
3,The punishment for heresy was being burned at the stake.,0,10
4,No it doesn't. Sex is defined by DNA. DNA cannot be changed from male to female. A sex change is putting lipstick on a pig. It may look different but it's still a pig,0,31


In [44]:
ilong_true = df.loc[(df["length"] > 300) & (df["length"] < 400) & (df['isAdHominem'] == 1)].sample(n=1).index[0]
ilong_false = df.loc[(df["length"] > 300) & (df["length"] < 400) & (df['isAdHominem'] == 0)].sample(n=1).index[0]
imed_true = df.loc[(df["length"] > 100) & (df["length"] < 150) & (df['isAdHominem'] == 1)].sample(n=1).index[0]
imed_false = df.loc[(df["length"] > 100) & (df["length"] < 150) & (df['isAdHominem'] == 0)].sample(n=1).index[0]
ishort_true = df.loc[(df["length"] > 10)  & (df["length"] < 20)  & (df['isAdHominem'] == 1)].sample(n=1).index[0]
ishort_false = df.loc[(df["length"] > 10)  & (df["length"] < 20)  & (df['isAdHominem'] == 0)].sample(n=1).index[0]
print("The indexes for the examples picked are {}, {}, {}, {}, {} and {}.".format(ilong_true, ilong_false, imed_true, imed_false, ishort_true, ishort_false))

The indexes for the examples picked are 13816, 18153, 14708, 18461, 15940 and 60407.


In [45]:
indexes = [ilong_true, ilong_false, imed_true, imed_false, ishort_true, ishort_false]
df_samples = df.loc[indexes,:]

## Cleaning the dataset

In [46]:
df_samples["body"] = df_samples["body"].apply(lambda x: " ".join(simple_preprocess(str(x), deacc=True)))
df_samples = df_samples.reset_index(drop=True).reindex(["length", "body", "isAdHominem"], axis=1)
df_samples

,length,body,isAdHominem
0,326,lying is the act of intentionally saying something that you know is wrong it is not saying something that you think is true but might be wrong so no it not basically lying anyway can find the tribes was thinking about know saw some documentary about it way back but can for the life of me remember what it was called but found something called koteka it basically penis sheath and is worn in some tribes on new guinea while it still covers the penis it only covers the penis not the scrotum and not the skin around the penis it also looks like penis though often very long go figure the same concept can also be found in tropical africa and northern south america it possible the documentary saw was about people with penis gourds in other news just remembered one somewhat famous culture in which people don cover their genitals nudism there are also records of casual nudity as in non sexual during certain festivals as well as exercise in th century bce sparta and of course ancient greek olympic athletes competed naked tl dr not lying might ve been wrong but by definition wasn lying some tribes in africa south america and oceania wear penis gourds and casual nudity isn unheard of neither in history nor today edit also what do you mean that your point you never tried to make the point that don know if there are cultures around the world that don cover their genitals you someone else said that no part of the human body is inherently sexy to which replied that the penis and vagina are inherently sexy but that that still doesn necessarily mean we should cover them as there are cultures around the world that don though ll admit was wrong that there are plenty of these you then asked for examples you never tried to make point at least not to me edit couple words,1
1,362,voted republican in the past elections so since you re simply asking for explanations of these views ll offer few can say don care at all what people do in the privacy of their own homes but also don believe the government should be in the business of defining what marriage is one way or the other the concept of marriage is deeply tied to cultural and religious values which obviously vary drastically and will constantly be butting up against whatever legal definition we give it today but realize the tax benefits are an issue and don really know how to solve it most importantly though this is not deal breaker for me nor is it for almost any other republican personally know you re right it stupid most republicans know agree on immigration first we have to be with any claims about how many illegal immigrants are actually here seeing as there are pretty big discrepancies among the data but assuming pew is correct it doesn really get to the heart of the issue as trump supporter what see from majority of the people on the left is an unwillingness to admit that we should strive to eliminate illegal immigration and that this has to start with enforcement of the current law of the land see people demonized for simply acknowledging million illegals receiving many benefits that us citizens is definitely not ideal it will suck for some people to be denied entry but the intention is to bring them over legally not keep them out forever don really care about the wall care that our current laws are enforced the majority people recently who have stated this clearly and unambiguously are republicans but your argument here seems to be illegal immigration is decreasing faster than most republicans acknowledge and to that would say yes you are right will not get into the statistical claims because the debate see is about whether or not we should treat immigration to the usa as right or privilege people holding up nowall signs are most often against the idea of simply keeping people out not that the wall costs too much in my experience,0
2,112,you don seem to have any true concept of what hardship is people endure all kinds of hardships for variety of reasons work mult

## Neural Network/TFIDF
From [here](../02_tfidf/neural_network.ipynb).

In [47]:
from keras import utils
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

vocab_size = 3000

tokenize = text.Tokenizer(num_words=vocab_size)
#tokenize.fit_on_texts(result.headline_text)

tokenize.fit_on_texts(df_samples["body"]) # only fit on train
x_test = tokenize.texts_to_matrix(df_samples["body"])
x_test.shape

(6, 3000)

In [48]:
# load json and create model
json_file = open('../02_tfidf/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../02_tfidf/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(x_test, df_samples['isAdHominem'], verbose=0)
ynew = loaded_model.predict_classes(x_test)

Loaded model from disk


In [49]:
df_samples["NN/TFIDF"] = ynew
df_samples

,length,body,isAdHominem,NN/TFIDF
0,326,lying is the act of intentionally saying something that you know is wrong it is not saying something that you think is true but might be wrong so no it not basically lying anyway can find the tribes was thinking about know saw some documentary about it way back but can for the life of me remember what it was called but found something called koteka it basically penis sheath and is worn in some tribes on new guinea while it still covers the penis it only covers the penis not the scrotum and not the skin around the penis it also looks like penis though often very long go figure the same concept can also be found in tropical africa and northern south america it possible the documentary saw was about people with penis gourds in other news just remembered one somewhat famous culture in which people don cover their genitals nudism there are also records of casual nudity as in non sexual during certain festivals as well as exercise in th century bce sparta and of course ancient greek olympic athletes competed naked tl dr not lying might ve been wrong but by definition wasn lying some tribes in africa south america and oceania wear penis gourds and casual nudity isn unheard of neither in history nor today edit also what do you mean that your point you never tried to make the point that don know if there are cultures around the world that don cover their genitals you someone else said that no part of the human body is inherently sexy to which replied that the penis and vagina are inherently sexy but that that still doesn necessarily mean we should cover them as there are cultures around the world that don though ll admit was wrong that there are plenty of these you then asked for examples you never tried to make point at least not to me edit couple words,1,1
1,362,voted republican in the past elections so since you re simply asking for explanations of these views ll offer few can say don care at all what people do in the privacy of their own homes but also don believe the government should be in the business of defining what marriage is one way or the other the concept of marriage is deeply tied to cultural and religious values which obviously vary drastically and will constantly be butting up against whatever legal definition we give it today but realize the tax benefits are an issue and don really know how to solve it most importantly though this is not deal breaker for me nor is it for almost any other republican personally know you re right it stupid most republicans know agree on immigration first we have to be with any claims about how many illegal immigrants are actually here seeing as there are pretty big discrepancies among the data but assuming pew is correct it doesn really get to the heart of the issue as trump supporter what see from majority of the people on the left is an unwillingness to admit that we should strive to eliminate illegal immigration and that this has to start with enforcement of the current law of the land see people demonized for simply acknowledging million illegals receiving many benefits that us citizens is definitely not ideal it will suck for some people to be denied entry but the intention is to bring them over legally not keep them out forever don really care about the wall care that our current laws are enforced the majority people recently who have stated this clearly and unambiguously are republicans but your argument here seems to be illegal immigration is decreasing faster than most republicans acknowledge and to that would say yes you are right will not get into the statistical claims because the debate see is about whether or not we should treat immigration to the usa as right or privilege people holding up nowall signs are most often against the idea of simply keeping people out not that the wall costs too much in my experience,0,1
2,112,you don seem to have any true concept of what hardship is people endure all kinds of hardships for variety of reas

## Neural Network/Word2Vec
From [here](../02_tfidf/tfidf.ipynb).

## LinearSVC/Word2Vec
From [here](../02_tfidf/tfidf.ipynb).

In [ ]:
## LinearSVC/TFIDF
From [here](../05)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split( df, test_size=0.3, random_state=1)

print("In total, the train contains", sum(train["isAdHominem"] == True), "ad hominems")
print("In total, the test contains", sum(test["isAdHominem"] == True), "ad hominems")

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
print("importing word2vec")
wv_from_bin = KeyedVectors.load_word2vec_format("~/notebooks/word2vec-GoogleNews-vectors/GoogleNews-vectors-negative300.bin.gz", binary=True)  # C binary format
print("imported word2vec")

In [ ]:
def combineData( dataset, length ):  
    paragraphRepresentations = np.zeros((len(dataset), maxLength, 300))
    
    for i, paragraph in enumerate(dataset["body"]):
        if (100* i / dataset.shape[0] % 10 == 0):
            print("{} of {} ({} %)\r".format(i, dataset.shape[0], 100* i / dataset.shape[0]))
        
        # Enumerate over the words in the tags-array (col 0 = words, col 1 = POS tags)
        for j, word in enumerate(simple_preprocess(paragraph, deacc=True)):
            #print("{}: {} ({})".format(i, j, word))
            if word.lower() in wv_from_bin:
                paragraphRepresentations[i, j] = wv_from_bin[word.lower()] 
    return paragraphRepresentations

In [ ]:
maxLength = df["length"].max() 
print("Start train data")
train_vectors = combineData(train, maxLength)

In [ ]:
test_vectors = combineData(test, maxLength)

In [ ]:
train_vectors.shape

In [ ]:
test_vectors.shape

In [ ]:
batch_size = 128
epochs = 10

## Fix imbalance

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train["isAdHominem"]),
                                                 train["isAdHominem"])

In [ ]:
class_weights

In [ ]:
from keras.models import Sequential
from keras.layers import Masking
from keras.layers import LSTM, Bidirectional, GRU
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

model = Sequential()
model.add(Masking(mask_value=0., input_shape=(maxLength, 300)))
model.add(Bidirectional(GRU(10, return_sequences=True), merge_mode='concat'))
model.add(Bidirectional(GRU(10), merge_mode='concat'))
model.add(Dense(64))
model.add(Dense(2, activation="softmax"))
#model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_vectors, np.array([train["isAdHominem"], 1-train["isAdHominem"]]).T,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    class_weight=class_weights)

In [ ]:
test_predicted = model.predict(test_vectors)

In [ ]:
model.save_weights("mode06.h5")

## Finally evaluate loss on train and validation set

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()